In [18]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [19]:
df = pd.read_csv("DFF_result.csv")

# Clean data.

# Pulse width values for Pass cases & Current percentage values for Fail cases. 
df.loc[df['fail'] == 'pass', 'Junction Values'] = df.loc[df['fail'] == 'pass', 'Pulse width info']
df.loc[df['fail'] == 'fail', 'Junction Values'] = df.loc[df['fail'] == 'fail', 'Current percentage']


# One-hot encode pass/fail.
drop_col = ['cus', 'real_flux', 'Faulty Junction', 'Pulse magnitude info', 'Phase state info', 'Pulse width info', 'Current percentage' ]
df = df.drop(drop_col, axis=1)

df.head()

,config,pattern,MC,fail,real_output,gold_output,Junction Values
0,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.0,0.04,-0.04",pass,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.83, 'B01RX2': 0.72, 'B01TX1': 0.6..."
1,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.01,-0.04",fail,"{'n300': '0000000000000000', 'n200': '00100000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.85, 'B01RX2': 0.73, 'B01TX1': 0.7..."
2,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.01,0.01",pass,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.79, 'B01RX2': 0.69, 'B01TX1': 0.6..."
3,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.06,0.05",pass,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.75, 'B01RX2': 0.66, 'B01TX1': 0.6..."
4,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.07,0.03",pass,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.77, 'B01RX2': 0.67, 'B01TX1': 0.6..."


In [20]:
# Rule 1: Initial Bias is between 60%-80%. 
# Rule 2: Junction 4 bias <55%. 


# Create rule 1, rule 2, pass/fail truth table.
for val in df['Junction Values']:
    val = eval(str(val))
    if 0.6 <= (val['B01RX1']) <= 0.8:
        df['Rule 1'] = 1
    else:
        df['Rule 1'] = 0
    if val['B4'] <0.55:
        df['Rule 2'] = 1
    else:
        df['Rule 2'] = 0

columns = df.columns.tolist()
columns.append(columns.pop(3))
df["fail"] = df["fail"].replace({"pass": 1, "fail": 0})
df = df[columns]

df.head()

,config,pattern,MC,real_output,gold_output,Junction Values,Rule 1,Rule 2,fail
0,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.0,0.04,-0.04","{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.83, 'B01RX2': 0.72, 'B01TX1': 0.6...",1,1,1
1,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.01,-0.04","{'n300': '0000000000000000', 'n200': '00100000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.85, 'B01RX2': 0.73, 'B01TX1': 0.7...",1,1,0
2,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.01,0.01","{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.79, 'B01RX2': 0.69, 'B01TX1': 0.6...",1,1,1
3,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.06,0.05","{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.75, 'B01RX2': 0.66, 'B01TX1': 0.6...",1,1,1
4,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.07,0.03","{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.77, 'B01RX2': 0.67, 'B01TX1': 0.6...",1,1,1


In [21]:
# Create buckets.
condition0 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 0, 0]).all(axis=1)
condition0_df = df[condition0]
print(len(condition0_df))

condition1 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 0, 1]).all(axis=1)
condition1_df = df[condition1]
print(len(condition1_df))

condition2 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 1, 0]).all(axis=1)
condition2_df = df[condition2]
print(len(condition2_df))

condition3 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 1, 1]).all(axis=1)
condition3_df = df[condition3]
print(len(condition3_df))

condition4 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 0, 0]).all(axis=1)
condition4_df = df[condition4]
print(len(condition4_df))

condition5 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 0, 1]).all(axis=1)
condition5_df = df[condition5]
print(len(condition5_df))

condition6 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 1, 0]).all(axis=1)
condition6_df = df[condition6]
print(len(condition6_df))

condition7 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 1, 1]).all(axis=1)
condition7_df = df[condition7]
print(len(condition7_df))

0
0
0
0
0
0
16713
23288


In [22]:
condition6_df['driver'] = condition6_df['config'].str.split('_').str[0]
condition6_df['load'] = condition6_df['config'].str.split('_').str[6]
condition6_df['splitterNum'] = condition6_df['config'].str.split('_').str[2]


C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_41272\1653661052.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condition6_df['driver'] = condition6_df['config'].str.split('_').str[0]
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_41272\1653661052.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condition6_df['load'] = condition6_df['config'].str.split('_').str[6]
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_41272\1653661052.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [23]:
condition6_df.head()

,config,pattern,MC,real_output,gold_output,Junction Values,Rule 1,Rule 2,fail,driver,load,splitterNum
1,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.01,-0.04","{'n300': '0000000000000000', 'n200': '00100000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.85, 'B01RX2': 0.73, 'B01TX1': 0.7...",1,1,0,and,and,0
12,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,0.07,-0.05","{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.86, 'B01RX2': 0.73, 'B01TX1': 0.7...",1,1,0,and,and,0
16,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.02,-0.02,-0.03","{'n300': '0000000000000000', 'n200': '00100000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.85, 'B01RX2': 0.73, 'B01TX1': 0.7...",1,1,0,and,and,0
17,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.02,-0.06,-0.04","{'n300': '0000000000000000', 'n200': '00100000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.86, 'B01RX2': 0.74, 'B01TX1': 0.7...",1,1,0,and,and,0
19,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.02,-0.08,-0.02","{'n300': '0000000000000000', 'n200': '00100000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.84, 'B01RX2': 0.72, 'B01TX1': 0.6...",1,1,0,and,and,0


In [24]:
condition6_df = pd.get_dummies(condition6_df, columns = ["driver", "load", "splitterNum"])
condition6_df

,config,pattern,MC,real_output,gold_output,Junction Values,Rule 1,Rule 2,fail,driver_and,...,driver_xor,load_and,load_dff,load_inv,load_or,load_xor,splitterNum_0,splitterNum_1,splitterNum_2,splitterNum_3
1,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.01,-0.04","{'n300': '0000000000000000', 'n200': '00100000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.85, 'B01RX2': 0.73, 'B01TX1': 0.7...",1,1,0,1,...,0,1,0,0,0,0,1,0,0,0
12,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,0.07,-0.05","{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.86, 'B01RX2': 0.73, 'B01TX1': 0.7...",1,1,0,1,...,0,1,0,0,0,0,1,0,0,0
16,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.02,-0.02,-0.03","{'n300': '0000000000000000', 'n200': '00100000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.85, 'B01RX2': 0.73, 'B01TX1': 0.7...",1,1,0,1,...,0,1,0,0,0,0,1,0,0,0
17,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.02,-0.06,-0.04","{'n300': '0000000000000000', 'n200': '00100000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.86, 'B01RX2': 0.74, 'B01TX1': 0.7...",1,1,0,1,...,0,1,0,0,0,0,1,0,0,0
19,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.02,-0.08,-0.02","{'n300': '0000000000000000', 'n200': '00100000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.84, 'B01RX2': 0.72, 'B01TX1': 0.6...",1,1,0,1,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39996,xor_spb_3_dff_spa_3_xor,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","0.16,0.03,-0.01","{'n300': '0000010000010100', 'n200': '00001000...","{'n300': '0001000001010011', 'n200': '00010000...","{'B01RX1': 0.64, 'B01RX2': 0.58, 'B01TX1': 0.5...",1,1,0,0,...,1,0,0,0,0,1,0,0,0,1
39997,xor_spb_3_dff_spa_3_xor,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","0.16,0.04,-0.01","{'n300': '0000010000010100', 'n200': '00001000...","{'n300': '0001000001010011', 'n200': '00010000...","{'B01RX1': 0.64, 'B01RX2': 0.58, 'B01TX1': 0.5...",1,1,0,0,...,1,0,0,0,0,1,0,0,0,1
39998,xor_spb_3_dff_spa_3_xor,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","0.16,0.09,-0.01","{'n300': '0000010000010100', 'n200': '00001000...","{'n300': '0001000001010011', 'n200': '00010000...","{'B01RX1': 0.64, 'B01RX2': 0.58, 'B01TX1': 0.5...",1,1,0,0,...,1,0,0,0,0,1,0,0,0,1
39999,xor_spb_3_dff_spa_3_xor,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","0.16,0.09,0.02","{'n300': '0000010000010100', 'n200': '00001000...","{'n300': '0001000001010011', 'n200': '00010000...","{'B01RX1': 0.61, 'B01RX2': 0.56, 'B01TX1': 0.5...",1,1,0,0,...,1,0,0,0,0,1,0,0,0,1


In [25]:
import Quine_McCluskey_Functions as qm